In [1]:
import os
import pickle
from scipy import signal
import libfmp.b
import librosa
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm
from load_djembe_marker import *
from foot_module import onset_calculations, onset_extraction, onset_filtering, utils, onset_plot
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, argrelmin, argrelmax
from pydub import AudioSegment
from scipy.signal import find_peaks
from pydub.generators import Triangle
import mir_eval
from compute_tempo_pos import *
from mutils import DataHandler
from dance_evaluation import *

In [ ]:
mvnfiles = ["BKO_E1_D1_01_Suku_T.mvnx", "BKO_E1_D1_02_Maraka_T.mvnx", "BKO_E1_D1_03_Wasulunka_T.mvnx", "BKO_E1_D2_04_Maraka_T.mvnx"]

filename = mvnfiles[2]
piece_name = os.path.basename(filename).split(".")[0]
data_handler = DataHandler()
motion_data, drum_onsets, start_f, end_f, start_t, end_t, cycle_onsets, beat_ref, bpm = data_handler.load_and_process_data(filename, mode = 'gr', drum = 'J2', section_idx=0)

### Two Sensor (Velocity)

In [ ]:
# SEGMENT_HEAD  SEGMENT_PELVIS   SEGMENT_T8  SEGMENT_LEFT_HAND  SEGMENT_LEFT_FOOT
pkl_filelist = os.listdir(f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/Dataset_V2")
data_handler = DataHandler()

result = {
    "piece": [],
    "bpm": [],
    "X_a": [],
    "Y_a": [],
    "Z_a": [],
    "bpm_mode": [],
    "bpm_median": [],
}

for idx, filename in enumerate(pkl_filelist):

    filename = filename.replace("_Dancers.csv", "_T.mvnx")
    piece_name = os.path.basename(filename).split(".")[0]
    try:
        motion_data, drum_onsets, start_f, end_f, start_t, end_t, cycle_onsets, beat_ref, bpm = data_handler.load_and_process_data(filename, mode = 'gr', drum = 'J2', section_idx=0)
    except ValueError as e:
        # Handle the error and continue
        print(f"Error encountered for: {e}")
        continue

    duration = int(end_t-start_t)
    w_sec = int(duration)
    h_sec = int(w_sec/4)

    mocap_fps = 240
    window_size = int(240*w_sec)
    hop_size = int(240*h_sec)
    tempi_range = np.arange(70,145,1)

    sensorA_seg_name = 'SEGMENT_RIGHT_FOOT'
    sensorB_seg_name = 'SEGMENT_LEFT_FOOT'

    sensorA_pos_data = motion_data['position'][sensorA_seg_name][start_f:end_f, :]    # size (n, 3)    [start_f:end_f,:]
    sensorB_pos_data = motion_data['position'][sensorB_seg_name][start_f:end_f, :]    # size (n, 3)    [start_f:end_f,:]

    sensorA_position = detrend_signal_array(sensorA_pos_data, cutoff= 0.5)
    sensorB_position = detrend_signal_array(sensorB_pos_data, cutoff= 0.5)

    novelty_length = len(sensorA_position)
    time_axis = np.arange(novelty_length)/240
    bpm_axes = []
    mag_axes = []

    mode= "zero_bi"
    for ax in range(3):
        
        posA_min, posA_max = np.min(sensorA_position[:, ax]), np.max(sensorA_position[:, ax])
        sensorA_position_norm = (
                2*(sensorA_position[:, ax] - posA_min) / (posA_max - posA_min) - 1
                if posA_max != posA_min 
                else np.zeros_like(sensorA_position[:, ax])
            )

        posB_min, posB_max = np.min(sensorB_position[:, ax]), np.max(sensorB_position[:, ax])
        sensorB_position_norm = (
                2*(sensorB_position[:, ax] - posB_min) / (posB_max - posB_min) - 1
                if posB_max != posB_min
                else np.zeros_like(sensorB_position[:, ax])
            )
        
        
        tempo_json = main_two_sensor(sensorA_position_norm.reshape(-1,1), sensorB_position_norm.reshape(-1,1),  
                                    mocap_fps, window_size, hop_size, tempi_range,
                                    T_filter= 0.25, mode=mode)


        # sensorA_abs_vel = tempo_json["sensorA_abs_vel"]
        # sensorB_abs_vel = tempo_json["sensorB_abs_vel"]
        # sensorA_dir_change = tempo_json["sensorA_dir_change"]
        # sensorB_dir_change = tempo_json["sensorB_dir_change"]                         
        # sensorA_dir_change_f = tempo_json["sensorA_dir_change_f"]
        # sensorB_dir_change_f = tempo_json["sensorB_dir_change_f"]
                                                        
        # sensorAB_onsets = tempo_json["sensorAB_onsets"]

        # tempogram_ab = tempo_json["tempogram_ab"]
        # time_axis_seconds = tempo_json["time_axis_seconds"]
        # tempo_axis_bpm = tempo_json["tempo_axis_bpm"]

        tempo_data_maxmethod = tempo_json["tempo_data_maxmethod"]
        # tempo_data_weightedkernel = tempo_json["tempo_data_weightedkernel"]
        # tempo_data_combinedtempogram = tempo_json["tempo_data_combinedtempogram"]

        # Max method
        # Aestimated_beat_pulse = tempo_data_maxmethod["estimated_beat_pulse"]
        # Atempo_curve = tempo_data_maxmethod["tempo_curve"]
        # Atempo_curve_time_axis = tempo_data_weightedkernel["tempo_curve_time_axis"]

        bpmA_arr = tempo_data_maxmethod["bpm_arr"]
        magA_arr = tempo_data_maxmethod["mag_arr"]
        tempo_A = np.round(np.average(bpmA_arr), 2)

        bpm_axes.append(bpmA_arr)

        if ax == 0:
            result["piece"].append(piece_name)
            result["bpm"].append(np.round(bpm,2))
            result["X_a"].append(tempo_A)

        elif ax == 1:
            result["Y_a"].append(tempo_A)

        elif ax == 2:
            result["Z_a"].append(tempo_A)

        mag_axes.append(magA_arr)
        mag_axes_arr = np.column_stack(mag_axes)    # n by 3 array
        
    bpm_axes_arr = np.column_stack(bpm_axes)    # n by 3 array
    bpm_mode = stats.mode(bpm_axes_arr.flatten())[0]
    bpm_median = np.median(bpm_axes_arr.flatten())
    result["bpm_mode"].append(bpm_mode)
    result["bpm_median"].append(bpm_median)
    
# Weighted method
# Bestimated_beat_pulse = tempo_data_weightedkernel["estimated_beat_pulse"]
# Btempo_curve = tempo_data_weightedkernel["tempo_curve"]
# Btempo_curve_time_axis = tempo_data_weightedkernel["tempo_curve_time_axis"]

# both_sensor_onsets_b = np.sum(sensor_dir_change_onsets, axis=1)
# both_sensor_onsets_b = np.where(both_sensor_onsets_b > 0, 1,0)
# both_sensor_onsets_bfil = filter_dir_onsets_by_threshold(both_sensor_onsets_b.reshape(-1,1), threshold_s=0.15)
# both_sensor_onsets = both_sensor_onsets_bfil.flatten()   # Binary onsets

# dance_onset, estimated_beat_onset, drum_ref = data_handler.onsets_for_plotting(sensorAB_onsets, Aestimated_beat_pulse, novelty_length)
# dance_bpm = data_handler.calc_tempo_from_onsets(dance_onset)

results_df = pd.DataFrame(result)
csv_filename = f"./results/pos_2s/foot/peraxis/foot_peraxis_{mode}_70_145.csv"
results_df.to_csv(csv_filename, index=False)
print(f"Results saved to {csv_filename}")  

# print("bpm:", bpm)
# print("A:", tempo_A)

Loaded BKO_E1_D1_08_Suku_T.pkl
Error encountered for: Group 'gr' not found in the dataset.
Loaded BKO_E3_D5_03_Wasulunka_T.pkl
Total Sections: 1
Computing tempograms...
Computing max method...
Computing weighted method...
Computing tempograms...
Computing max method...
Computing weighted method...
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E2_D4_05_Sandia_T.pkl
Total Sections: 1
Computing tempograms...
Computing max method...
Computing weighted method...
Computing tempograms...
Computing max method...
Computing weighted method...
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E2_D4_01_Suku_T.pkl
Total Sections: 1
Computing tempograms...
Computing max method...
Computing weighted method...
Computing tempograms...
Computing max method...
Computing weighted method...
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E2_D4_03_Wasulunka_T.pkl
Total Sections: 1
Computin

In [ ]:
plot_tempogram_perAxis(tempo_json, islog= 'no', dpi=200)

In [ ]:
plt.figure(figsize=(40, 6), dpi=200)
plt.plot(100*Aestimated_beat_pulse[start_f:end_f], linewidth=1, color = 'b')
plt.plot(Atempo_curve[start_f:end_f], linewidth=1, color = 'r')

plt.xlabel('Time (seconds)')
plt.ylabel('Tempo (BPM)')
plt.title(f'{piece_name} Start:{round(start_t)} End:{round(end_t)}')
plt.grid(True)
plt.show()

In [ ]:
pos_b = np.where(sensorA_position[start_f:end_f,1]> 0, 1,0)

indices = np.where(sensorA_dir_change[start_f:end_f] > 0)[0]
filtered_indices = indices[pos_b[indices] == 1]

plt.figure(figsize=(40, 6), dpi=300)
# plt.plot(sensorA_position[start_f:end_f, 1], linewidth = 1.0, color='b')
plt.plot(pos_b, linewidth = 1.0, color='r')
plt.vlines(x= indices, ymin=0.0, ymax=1, color='g', linewidth=1.2,)

In [ ]:
# Per mode: drum onset and directional change onset plot

plt.figure(figsize=(40, 6), dpi=300)
window_size = 0.1
for onset in beat_ref:
    window_start = onset - (window_size/2)  # Start of the window (25ms before)
    window_end = onset + (window_size/2)   # End of the window (25ms after)
    
    # Plot shaded window
    plt.axvspan(window_start, window_end, color='red', alpha=0.2)
    # Plot reference onset as a vertical line
    plt.axvline(onset, color='red', linestyle='--', linewidth=0.8)

plt.vlines(x= dance_onset, ymin=0.0, ymax=1, color='g', linewidth=1.5,)
# plt.vlines(x= estimated_beat_onset, ymin=0.0, ymax=1, color='g', linewidth=1.5,)

plt.xlabel('Time (seconds)')
plt.title(f'{piece_name}')
plt.grid(True)
plt.show()

In [ ]:
results = evaluate_dance_onsets_with_half_beats(beat_ref, dance_onset, tolerance=0.15)
# results = evaluate_dance_onsets_with_half_beats(beat_ref, estimated_beat_onset, tolerance=0.15)

print("\nEvaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")
print("-"*40)

In [ ]:
# Evaluate using mir_eval
scores = mir_eval.beat.evaluate(drum_reference, beat_estimated)
precision, recall, f_measure = mir_eval.onset.f_measure(drum_reference, beat_estimated)
print(f'Precision: {precision}, Recall: {recall}, F-Measure: {f_measure}')

In [ ]:
#   Save audio plots

time = np.arange(novelty_length) / mocap_fps
peaks, _ = signal.find_peaks(Aestimated_beat_pulse)  # , prominence=0.02
beat_peaks_sec = time[peaks]


click_duration = 50  # milliseconds
click_freq = 1200  # Hz
file_name ="maraka_vel"
# Generate a single click sound
click = Triangle(click_freq).to_audio_segment(duration=click_duration)

onset_times = beat_peaks_sec  # kept_onsets/240   beat_peaks_sec
dN = novelty_length
total_duration = (dN/240)*1000  #  in milliseconds

audio = AudioSegment.silent(duration=total_duration)
for onset in onset_times:
    position = int(onset * 1000)  # Convert onset time to milliseconds
    audio = audio.overlay(click, position=position)

# Export the audio with clicks to a file
audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Both_Foot_new.wav"), format="wav")
# audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Bothhand_dir.wav"), format="wav")